# 유튜브에서 영상 관련 데이터 가져오기
## 썸네일 링크, 카테고리, 제목, 영상 길이, 조회수, 업로드 날짜, 좋아요수, 싫어요수

In [11]:
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

delay = 3
browser = Chrome("./chromedriver")
browser.implicitly_wait(delay)
browser.get("https://www.youtube.com/channel/UCuZu8NrpBG4WPXRi-hPBl-A/videos") # 유튜버의 동영상 페이지로 들어가기
browser.maximize_window()
body = browser.find_element_by_tag_name('body') # body태그 가져오기

num_of_pagedowns = 22 # 스크롤
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1 #video페이지의 스크롤을 충분히 밑으로 내려놓기

In [12]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')
video_list2 = html.select('div #dismissable') # 정보가 담겨있는 태그

youtube_url = 'http://www.youtube.com'

park_url = []

#썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요 를 저장하기 위한 리스트
thumbnail_list = []
category_list = []
upload_date_list = []
video_length_list = []
title_list = []
view_count_list = []
likes_count_list = []
dislikes_count_list = []

print("video_list2의 길이:",len(video_list2)) # 동영상 개수를 출력

for i in range(len(video_list2)):
    url = youtube_url + video_list2[i].find('a',{'id': 'thumbnail'})['href']
    park_url.append(url) # 모든 영상의 url을 youtube_url에 붙여서 완전한 url로 만들고 park_url이라는 리스트에 넣어줌

video_list2의 길이: 90


### 영상 길이 가져오기

In [13]:
for video in video_list2: 
    video_length = video.select_one("ytd-thumbnail-overlay-time-status-renderer.ytd-thumbnail > span.style-scope")
    video_length = video_length.text.replace("\n","").strip()
    video_length_list.append(video_length)
    print(video_length)

21:09
31:17
31:18
31:37
24:31
23:00
27:04
32:43
21:23
20:24
22:33
28:12
22:42
5:33
20:01
14:16
12:50
31:51
17:45
31:02
23:27
32:03
7:52
23:06
11:12
21:59
16:56
34:03
23:29
21:16
21:04
23:01
20:36
30:50
26:15
30:53
22:40
33:22
25:17
22:29
22:06
18:58
22:08
16:49
34:49
16:20
32:39
8:48
24:37
36:44
19:28
33:38
31:56
17:45
33:01
21:19
23:21
16:30
15:14
32:06
27:57
32:03
30:51
32:27
16:25
22:16
22:28
33:25
29:36
22:28
21:45
32:22
25:41
31:33
23:54
21:31
31:54
21:01
24:38
3:58
28:57
19:44
32:35
27:32
24:03
34:18
5:50
23:24
29:05
21:08


### 나머지 데이터 가져오기
#### try except 문을 사용하여 예외처리 해주기 _ 해당 태그가 공백인 경우
#### ex) 좋아요 수 비공개, 더보기란 공란

In [14]:
from selenium.common.exceptions import ElementNotInteractableException

for i in range(len(video_list2)):
    try:
        # video페이지에서 수집할 수 있는 썸네일, 제목을 가져오기
        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 찾고 거기서 src링크 가져오기 # 1
        title = video_list2[i].find('a',{'id':'video-title'}).text 

        # 다음 영상을 클릭해서 들어간 페이지 
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩 대기시간
        
        browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기 버튼 클릭
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링 
        category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
        view_count = html.find('span', {'class': 'view-count'}).text 
        # <파이썬 정규 표현식을 지원하는 >re모듈을 이용
        likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', 'aria-label':re.compile('좋아요')}).text 
        dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', 'aria-label':re.compile('싫어요')}).text 
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text 
        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
   
    
    except ElementNotInteractableException: # 더보기 란이 없을 경우 예외처리

        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져오기 
        title = video_list2[i].find('a',{'id':'video-title'}).text 

        # 다음 영상을 클릭해서 들어간 페이지 
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩 대기시간
        
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링
        category = html.select_one("div#always-shown div#content a.style-scope")
        view_count = html.find('span', {'class': 'view-count'}).text 
        # <파이썬 정규 표현식을 지원하는 >re모듈을 이용
        likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', 'aria-label':re.compile('좋아요')}).text
        dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', 'aria-label':re.compile('싫어요')}).text
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8

        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)

    except AttributeError: # 좋아요 싫어요 수가 비공개일 경우 예외처리 _ 
        
        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
        title = video_list2[i].find('a',{'id':'video-title'}).text # 2 

        # 여기서부터 영상 클릭해서 들어간 페이지
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
        # body = browser.find_element_by_tag_name('body')
        browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링
        category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
        view_count = html.find('span', {'class': 'view-count'}).text

        # 좋아요, 싫어요 수 비공개이기 때문에 0 으로 설정 (나중에 데이터 처리할 때 평균값으로 대체)
        likes_count = "0"
        dislikes_count = "0"
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text
        
        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    
    
    finally:    
        thumbnail_list.append(thumbnail)
        title_list.append(title)
        category_list.append(category)
        view_count_list.append(view_count)
        likes_count_list.append(likes_count)
        dislikes_count_list.append(dislikes_count)
        upload_date_list.append(upload_date)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='more']/yt-formatted-string"}
  (Session info: chrome=79.0.3945.117)


In [ ]:
print(len(thumbnail_list),len(video_length_list),len(title_list))
import copy
video_length_list_copy = copy.deepcopy(video_length_list[:len(thumbnail_list)])
len(video_length_list_copy), len(video_length_list)

### pandas 를 이용해 가져온 정보로 표 만들기

In [ ]:
import pandas as pd

insert_data = pd.DataFrame({
                    'thumbnail': thumbnail_list,
                    'title' : title_list,
                    'category': category_list,
                    'view_count': view_count_list,
                    'likes_count': likes_count_list,
                    'dislikes_count':dislikes_count_list,
                    'upload_date': upload_date_list,
                    'video_length' : video_length_list_copy
                    }) 
insert_data

### 데이터 엑셀화

In [10]:
import openpyxl
insert_data.to_excel("조효진.xlsx",encoding="utf8")